## Telecom Churn Case study

### Business Problem
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, ___retaining high profitable customers is the number one business goal___.

To reduce customer churn, __telecom companies need to predict which customers are at high risk of churn__.

In this project, we will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

 __Importing Required Libraries__

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as w
w.filterwarnings('ignore')
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)

In [ ]:
data=pd.read_csv('telecom_churn_data.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

As checked above, there are 214 numeric columns and 12 non-numeric columns

In [ ]:
# look at data statistics
data.describe(include='all')

#### In churn prediction, we assume that there are three phases of customer lifecycle :

The ‘good’ phase [Month 6 & 7]<br>
The ‘action’ phase [Month 8]<br>
The ‘churn’ phase [Month 9]<br><br>
In this case, since we are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month is the ‘churn’ phase.

### Data Preparation

Let us create some utility functions

In [ ]:
# Method for Checking missing values percentages
def checkMissingPercent(dataset, cutoff):
    missing = round(100*(dataset.isnull().sum()/dataset.shape[0]))
    return missing.loc[missing>cutoff]

In [ ]:
# Method for imputing data 
def imputeData(df, col_list):
    for i in [x + y for y in ['_6','_7','_8','_9'] for x in col_list]:
        df[i].fillna(0,inplace = True)

__Handling missing values__

In [ ]:
mod_data=data.copy()

In [ ]:
# Since mobile no has all unique values and represents a particular customer, it can be dropped from the dataset.
# Similarly, circle_id has all same values(109), it also can be dropped.
mod_data.drop(['mobile_number', 'circle_id'], axis=1, inplace=True)

In [ ]:
# look at missing value ratio in each column
checkMissingPercent(mod_data, 0)

As checked above, there are so many columns conatining missing values. Among them, there are some columns which has more than 70% of missing values. We will not directly delete those columns. Let us first check that these values as null because of no transactions or because of some other reason.

In [ ]:
# getting all columns for month of June which has 75% missing values
cols = checkMissingPercent(mod_data, 74).index

mod_data.loc[mod_data.date_of_last_rech_data_6.isna(),cols].head()

As checked above, all the columns has null values where date of last recharge is missing. This is valid, we can replace these null values with 0 as there is no recharge done.

In [ ]:
# imputing all the columns other than those containg date with 0 having more than 50% missing value
cols = list(filter(lambda x : not x.startswith('date') , checkMissingPercent(mod_data, 50).index))

mod_data[cols]=mod_data[cols].apply(lambda x: x.fillna(0))
mod_data[cols].head()

Checking again percent of missing values

In [ ]:
checkMissingPercent(mod_data, 0)

Let us have a look at the non-numeric columns

In [ ]:
obj=mod_data.select_dtypes(include='object')
for i in obj.columns:
    print(i,'', obj[i].nunique(),'', obj[i].isna().sum()) 

We have already used date to fill the missing values. Further these date columns seems to be irrelevant in our analysis, so we will drop these columns

In [ ]:
mod_data = mod_data.drop(obj.columns, axis=1)

Again checking for the missing values

In [ ]:
checkMissingPercent(mod_data, 0)

In [ ]:
cols=list(checkMissingPercent(mod_data, 0).index)
mod_data[cols].describe()

As checked above, all the columns have their minimum value 0, but since the missing percent is very low around 4-5%, this can be because of technical or human error, its better to fill these values with median rather than 0. 

In [ ]:
# filling the columns above with median
mod_data[cols]=mod_data[cols].apply(lambda x: x.fillna(x.median()))
mod_data[cols].head()

Checking if our missing value imputation is successfully done or not

In [ ]:
all(mod_data.isna().sum()==0)

In [ ]:
# removing duplicates from row
mod_data.drop_duplicates(inplace=True)
mod_data.shape

There are a few columns whose names are not consistent with other columns. Let make them same.

In [ ]:
print(list(filter(lambda x: x[-1].isalpha(), mod_data.columns)))
mod_data.rename(columns={'aug_vbc_3g':'vbc_3g_8', 'jul_vbc_3g':'vbc_3g_7', 'jun_vbc_3g':'vbc_3g_6',
                         'sep_vbc_3g':'vbc_3g_9'}, inplace=True)
mod_data.head()

__Taking only the data of high valued customer by taking average of total recharge amount of good months__

In [ ]:
mod_data['av_rech_amt_6_7']=((mod_data.av_rech_amt_data_6 * mod_data.total_rech_data_6 + mod_data.total_rech_amt_6)+
                             (mod_data.av_rech_amt_data_7 * mod_data.total_rech_data_7 + mod_data.total_rech_amt_7)) / 2

# mod_data.drop(['av_rech_amt_data_6','total_rech_data_6','total_rech_amt_6','av_rech_amt_data_7',
#                'total_rech_data_7','total_rech_amt_7'], axis=1, inplace=True)


high_value_cust = mod_data[mod_data.av_rech_amt_6_7>mod_data.av_rech_amt_6_7.quantile(0.7)]
len(high_value_cust)

In [ ]:
high_value_cust.shape

**Tagging the churned customers (churn=1, else 0) based on the fourth month as follows: Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. The attributes you need to use to tag churners are:**
<br>
    1. total_ic_mou_9
    2. total_og_mou_9
    3. vol_2g_mb_9
    4. vol_3g_mb_9

In [ ]:
high_value_cust['churn'] = (high_value_cust.total_ic_mou_9+high_value_cust.total_og_mou_9 + high_value_cust.vol_3g_mb_9 + high_value_cust.vol_2g_mb_9).apply(lambda x: 1 if x==0 else 0)
high_value_cust.head()

In [ ]:
high_value_cust.churn.value_counts()

In [ ]:
print('churn rate:', round((2433/27520)*100,2), '%')

Our dataset has high class imbalance, we will take care of it while building a model.

Removing all the attributes corresponding to the churn phase (all attributes having ‘ _9’, etc. in their names).

In [ ]:
high_value_cust.drop(list(filter(lambda x: x[-1]=='9',high_value_cust.columns)), axis=1, inplace=True)
high_value_cust.head()

In [ ]:
high_value_cust.shape

As checked in the data dictionary, columns start with fb and night are schemes which are used for facebook and night packs respectively, so they are categorical columns(yes/no). Same as with churn columns. We will convert then to object type. This will help in doing EDA. 

In [ ]:
cols=list(filter(lambda x: x.startswith('fb') or x.startswith('night'), high_value_cust.columns))
cols

In [ ]:
cols.append('churn')
high_value_cust[cols]=high_value_cust[cols].astype('object')

### Exploratory Data Analysis

Let us create some utility functions

In [ ]:
# Method to add or subtract 2 columns to form a new column based on a pattern provided.
    # col_a_end_str - pattern to search from end of column A
    # col_b_end_str - pattern to search from end of column B
    # avg_or_diff - 'avg' for average and 'diff' for subtraction of 2 columns
    # new_name_end_str - end pattern to give to new columns
    # dataframe - a dataframe   

def addOrSubColumns(col_a_end_str, col_b_end_str, avg_or_diff, new_name_end_str, dataframe):
    li=[]

    s=set(filter( lambda x: x[-len(col_a_end_str):]==col_a_end_str, dataframe.select_dtypes(exclude='object').columns))
    s1=set(filter( lambda x:  x[-len(col_b_end_str):]==col_b_end_str, dataframe.select_dtypes(exclude='object').columns))

    for i in list(s):
        k=i[:-len(col_a_end_str)]
        a=k+col_a_end_str
        b=k+col_b_end_str
        if  b in s1:
            if avg_or_diff=='diff':
                dataframe[k+new_name_end_str]= (dataframe[b] - dataframe[a])
            else:
                dataframe[k+new_name_end_str]= (dataframe[b] + dataframe[a])/2
            li+=[a,b]
            s.remove(a); s1.remove(b)
        
    return dataframe.drop(li, axis=1)

In [ ]:
# method to capp outliers
def cappingOutliers(dataframe, lower_quantile, upper_quantile, columns, cap=False):
    for i in columns:
        print('outliers in',i, ':', len(dataframe[i][(dataframe[i]>dataframe[i].quantile(upper_quantile)) | 
              (dataframe[i]<dataframe[i].quantile(lower_quantile))]))
        if cap:
            dataframe[i][dataframe[i]>dataframe[i].quantile(upper_quantile)] = dataframe[i].quantile(upper_quantile)
            dataframe[i][dataframe[i]<dataframe[i].quantile(lower_quantile)] = dataframe[i].quantile(lower_quantile)

In [ ]:
def univariate(dataset,col,plt_type):
    #col = dataset.columns
    if plt_type=='single':
        plt.figure(figsize=(12, 6))
        if dataset[col].dtypes != 'object':
            sns.distplot(dataset[col])
            dataset[col].describe()
        else:
            sns.countplot(dataset[col])
            dataset[col].value_counts()
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.title( 'Frequency Plot of ' + str(col) , fontsize=12, fontweight=0, color='Blue')
    else:
        #plt.figure(figsize=(10, 5))
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
        col1, col2, col3 =col+'_6',col+'_7',col+'_8'
        if dataset[col1].dtypes != 'object':
            sns.distplot(dataset[col1], ax = ax1)
            sns.distplot(dataset[col2], ax = ax2)
            sns.distplot(dataset[col3], ax = ax3)
        else:
            sns.countplot(dataset[col1], ax = ax1)
            sns.countplot(dataset[col2], ax = ax2)
            sns.countplot(dataset[col3], ax = ax3)
        plt.subplots_adjust(wspace=0.5, hspace=0.5)
        fig.set_size_inches(13,5)
        fig.suptitle('Frequency Plot of ' + str(col), color = 'Blue')
    plt.show()

In [ ]:
# ---- Bivariate Analysis ---- #
def bivariate(dataset, col1, col2):
    plt.figure(figsize=(12, 6))
    if (dataset[col1].dtypes == 'object' and dataset[col2].dtypes != 'object'):
        sns.boxplot(x = col1, y = col2, data = dataset)
        plt.xlabel(col1)
        plt.ylabel(col2)
    elif (dataset[col1].dtypes != 'object' and dataset[col2].dtypes == 'object'):
        sns.boxplot(x = col2, y = col1, data = dataset)
        if max(dataset[col1])>10000:
            plt.yscale('log')
        plt.xlabel(col2)
        plt.ylabel(col1)
    plt.title( 'Box Plot of ' + str(col1)+ ' vs '+ str(col2) , fontsize=12, fontweight=0, color='Blue')
    plt.show()

In [ ]:
# A plot to show how a column vary in the month of June, July and August month against churn status.
def plot_vs_Churn(dataset,col):
    # per month churn vs Non-Churn
    fig, ax = plt.subplots(figsize=(7,4))
     
    colList=list(data.filter(regex=(col)).columns)
    colList = colList[:3]
    plt.plot(high_value_cust.groupby('churn')[colList].mean().T)
    ax.set_xticklabels(['Jun','Jul','Aug'])
    
    ## Add legend
    plt.legend(['Non-Churn', 'Churn'])
    
    # Add titles
    plt.title( str(col) +" V/S Month", fontsize=12, fontweight=0, color='orange')
    plt.xlabel("Month")
    plt.ylabel(col)
    plt.show()
    
    # Numeric stats for per month churn vs Non-Churn
    return high_value_cust.groupby('churn')[colList].mean()

In [ ]:
plot_vs_Churn(high_value_cust,'total_ic_mou')

__Observation__
1. Total incoming calls drops at a faster pace for the churners from the month of June to July.
2. For non-churners the graph is almost constant.

In [ ]:
plot_vs_Churn(high_value_cust,'total_og_mou')

__Observation__
1. Total outgoing calls drops significantly for the churners from the month of June to July. We could also see that churners were quite higher in number than non churners in making outgoing calls in the month of June.
2. For non-churners the graph is remains constant.

In [ ]:
plot_vs_Churn(high_value_cust,'total_rech_amt')

__Observation__
1. Total recharge amount drops significantly for the churners from the month of June to July. We have also observed that churners were quite spending higher amount in recharging than non churners in the month of June.
2. For non-churners the graph is almost constant.

In [ ]:
plot_vs_Churn(high_value_cust,'max_rech_amt')

__Observation__
1. Maximum recharge amount drops for the churners from the month of June to July and it dropped at a steep rate to August.
2. For non-churners the graph is almost constant.

In [ ]:
plot_vs_Churn(high_value_cust,'arpu')

__Observation__
1. Average Revenue Per User drops at a faster pace for the churners from the month of June to July.The ARPU from the churners was quite higher than the non-churners in the month of June.
2. While for non-churners the graph is almost constant.

In [ ]:
plot_vs_Churn(high_value_cust,'vol_2g_mb')

__Observation__
1. Usage volume of 2G data drops at a faster pace for the churners from the month of June to July.
2. While for non-churners the graph is significantly same.

In [ ]:
plot_vs_Churn(high_value_cust,'vol_3g_mb')

__Observation__
1. Usage volume of 3G data drops significantly for the churners from the month of June to July.
2. While for non-churners the graph is fairly same.

In [ ]:
# plot_vs_Churn(high_value_cust,'night_pck_user')

__Observation__
1. Night pack users drops significantly for the churners from the month of June to July.
2. For non-churners the graph is fairly constant.

In [ ]:
#After analysis we do not need these columns as we have got a derived column av_rech_amt_6_7
high_value_cust.drop(['av_rech_amt_data_6','total_rech_data_6','total_rech_amt_6','av_rech_amt_data_7',
               'total_rech_data_7','total_rech_amt_7'], axis=1, inplace=True)

In [ ]:
univariate(high_value_cust,'aon','single')

__Observation__<br>
The frequency of customers is highest for lower age on network while it gradually decreases then interestingly we could see a spike in frequency at 3200 around. 

In [ ]:
univariate(high_value_cust,'av_rech_amt_6_7','single')

__Observation__<br>
Most of the customers make smaller amount of recharges in the good phase.

In [ ]:
univariate(high_value_cust,'churn','single')

__Observation__<br>
We could clearly see the class imbalance here as the number of churns are far less than non churners.

In [ ]:
univariate(high_value_cust,'fb_user','multi')

__Observation__<br>
We could see from above frequency plots that from june to august the number of people purchasing fb_packs decreasing.

In [ ]:
univariate(high_value_cust,'night_pck_user','multi')

__Observation__<br>
From the above frequency plots it can be seen that from june to august the number of people purchasing night_packs is fairly constant.Hence, this variable is not much significant for our analysis.

In [ ]:
# Removing the night_pack columns anf fb_user columns
l=list(high_value_cust.select_dtypes(include='object').columns)
l.remove('churn')
high_value_cust.drop(l, axis=1, inplace=True)

In [ ]:
bivariate(high_value_cust,'aon','churn')

__Observation__<br>
From the above box plot it has been observed that age on network of non churners is more as compared to the churners.

In [ ]:
bivariate(high_value_cust,'av_rech_amt_6_7','churn')

__Observation__<br>
From the above box plot of average recharge amount of good phase months it can be seen that the average recharge amount is more for non churners than the churners.

#### Derived features

In [ ]:
# aggregating the columns of good months
high_value_cust = addOrSubColumns('6', '7', 'avg', '6_7', high_value_cust)

In [ ]:
# getting average recharge amount for action month
high_value_cust['av_rech_amt_8']=(high_value_cust.av_rech_amt_data_8 * high_value_cust.total_rech_data_8 + 
                                  high_value_cust.total_rech_amt_8)

high_value_cust.drop(['av_rech_amt_data_8','total_rech_data_8','total_rech_amt_8'], axis=1, inplace=True)

In [ ]:
# Finding difference between aggregated columns(6_7) and the action month columns
high_value_cust = addOrSubColumns('6_7', '8', 'diff', 'diff', high_value_cust)

Removing the numeric columns having more than 85% of values as a single value (highly skewed columns)

In [ ]:
li=[]
for i in high_value_cust.select_dtypes(exclude='object').columns:
    if max(high_value_cust[i].value_counts())/len(high_value_cust) >0.85:
        li.append(i)

high_value_cust.drop(li, axis=1, inplace=True)

In [ ]:
high_value_cust.shape

In [ ]:
high_value_cust.head()

__Outlier Treatment__

In [ ]:
round(high_value_cust.describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.9,0.95,0.99]),2)

In [ ]:
# call the function to check and cap the outliers
cols=list(high_value_cust.select_dtypes(exclude='object').columns) # columns to remove ouliers
cappingOutliers(high_value_cust, 0.01,0.99, cols, True)

__Analysis on Derived Features__

In [ ]:
bivariate(high_value_cust, 'total_og_mou_diff', 'churn')

In [ ]:
bivariate(high_value_cust, 'total_ic_mou_diff', 'churn')

In [ ]:
bivariate(high_value_cust,'arpu_diff','churn')

In [ ]:
bivariate(high_value_cust,'vol_3g_mb_diff','churn')

In [ ]:
bivariate(high_value_cust,'max_rech_data_diff','churn')

In [ ]:
bivariate(high_value_cust,'max_rech_amt_diff','churn')

In [ ]:
#Heat Map for checking correlation among variables
plt.figure(figsize=(25,20))
plt.title('Heat map for correlation', pad=20)
ax=sns.heatmap(high_value_cust.corr(), linewidth =0.3, center=0, annot=True, fmt='.2f')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

## Model Building

In [ ]:
# import required libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

# !pip install imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
high_value_cust.columns

In [ ]:
# converting churn to numeric data type
high_value_cust['churn'] = pd.to_numeric(high_value_cust['churn'])

In [ ]:
# divide data into train and test
X = high_value_cust.drop("churn", axis = 1)
y = high_value_cust.churn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [ ]:
# print shapes of train and test sets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

__Scaling the data__

In [ ]:
scaler =StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

__Handling the class imbalance__

In [ ]:
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [ ]:
y_train.value_counts()

Some Utility functions

In [ ]:
def getScores(confusion ):
    print('Confusion Matrix -')
    print(confusion)
    print('')
    TP = confusion[1,1] # true positive 
    TN = confusion[0,0] # true negatives
    FP = confusion[0,1] # false positives
    FN = confusion[1,0] # false negatives

    # Let's see the sensitivity of our logistic regression model
    print('sensitivity:',TP / float(TP+FN))
    print('')
    
    # Let us calculate specificity
    print('specificity:',TN / float(TN+FP))
    print('')
    

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = roc_curve( actual, probs, drop_intermediate = False )
    auc_score =roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()


### Model 1 - PCA with logistic regression

In [ ]:
# create PCA object
pca = PCA()

# fit to train data
X_train_pca = pca.fit_transform(X_train)

# look at explainded variance of PCA components
print(pd.Series(np.round(pca.explained_variance_ratio_.cumsum(), 4)*100))

In [ ]:
# plot cumulative variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
fig = plt.figure(figsize=[5,5])
plt.plot(cumulative_variance)
plt.ylabel("Cumulative variance explained")
plt.show()

In [ ]:
# create pipeline
pca_components = 30
steps = [
         ("pca", IncrementalPCA(n_components=pca_components)),
         ("logistic", LogisticRegression())
        ]
pipeline = Pipeline(steps)

In [ ]:
df_train_pca = pipeline.named_steps['pca'].fit_transform(X_train)
corrmat = np.corrcoef(df_train_pca.transpose())
plt.figure(figsize=[15,15])
sns.heatmap(corrmat, annot=True)

In [ ]:
# fit model
pipeline.fit(X_train, y_train)

# check score (mean accuracy score) on train data
pipeline.score(X_train, y_train)

# predict churn on test data
y_pred_train = pipeline.predict(X_train)

# create predicted probabilities on test data
y_pred_prob_train = pipeline.predict_proba(X_train)[:, 1]

# create confusion matrix
confusion = confusion_matrix(y_train, y_pred_train)

# get scores
getScores(confusion)

# print auc-roc score
print("AUC-ROC Score on train data:", round(roc_auc_score(y_train, y_pred_prob_train),2))

In [ ]:
draw_roc(y_train, y_pred_prob_train)

In [ ]:
# predict churn on test data
y_pred = pipeline.predict(X_test)

# create predicted probabilities on test data
y_pred_prob = pipeline.predict_proba(X_test)[:, 1]

# create confusion matrix
confusion = confusion_matrix(y_test, y_pred)

# get scores
getScores(confusion)

# print auc-roc score
print("AUC-ROC Score on test data:", round(roc_auc_score(y_test, y_pred_prob),2))

__Model Optimization by taking optimal cutoff value for logistic regression__

In [ ]:
logistic_df = pd.DataFrame({'y_train':y_train, 'y_pred_prob_train':y_pred_prob_train})
logistic_df.head()

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = np.arange(0.3,0.71,0.05)
print(numbers)
for i in numbers:
    logistic_df['cutoff_'+str(round(i,2))]= logistic_df.y_pred_prob_train.map(lambda x: 1 if x > i else 0)
logistic_df.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['accuracy','sensitivity','specificity'])

for i in logistic_df.iloc[:,2:].columns:
    confusion = confusion_matrix(logistic_df['y_train'], logistic_df[i])    
    speci = confusion[0,0]/(confusion[0,0]+confusion[0,1])
    sensi = confusion[1,1]/(confusion[1,0]+confusion[1,1])
    accuracy = (confusion[0,0]+confusion[1,1])/sum(sum(confusion))
    cutoff_df.loc[i] =[accuracy, sensi, speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(y=['accuracy','sensitivity','specificity'], figsize=(10,5))


### Model2 - Logistic Regression

In [ ]:
# # PCA
# pca = PCA()

# # logistic regression - the class weight is used to handle class imbalance - it adjusts the cost function
# logistic = LogisticRegression(class_weight='balanced')

# # create pipeline
# steps = [("scaler", StandardScaler()), 
#          ("pca", pca),
#          ("logistic", logistic)
#         ]

# # compile pipeline
# pca_logistic = Pipeline(steps)

# # hyperparameter space
# params = {'pca__n_components': [27, 37], 'logistic__C': [0.001, 0.1, 0.5, 1, 2, 3, 4, 5, 10], 'logistic__penalty': ['l1', 'l2']}

# # create gridsearch object
# model = GridSearchCV(estimator=pca_logistic, cv=5, param_grid=params, scoring='roc_auc', n_jobs=-1, verbose=1)

In [ ]:
# # fit model
# model.fit(X_train, y_train)

In [ ]:
# cross validation results
# pd.DataFrame(model.cv_results_)

In [ ]:
# # print best hyperparameters
# print("Best AUC: ", model.best_score_)
# print("Best hyperparameters: ", model.best_params_)

In [ ]:
# # predict churn on test data
# y_pred = model.predict(X_test)

# # create onfusion matrix
# confusion = confusion_matrix(y_test, y_pred)
# print(confusion)

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

# # Let's see the sensitivity of our logistic regression model
# print('sensitivity:',TP / float(TP+FN))

# # Let us calculate specificity
# print('specificity:',TN / float(TN+FP))

# # check area under curve
# y_pred_prob = model.predict_proba(X_test)[:, 1]
# print("AUC-ROC Score on test data:", round(roc_auc_score(y_test, y_pred_prob),2))

### Model2 - Random Forest

In [ ]:
# Create a based model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# predict churn on test data
y_pred = rf.predict(X_test)

# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)

getScores(confusion)

# check area under curve
y_pred_prob = model.predict_proba(X_test)[:, 1]
print("AUC-ROC Score on test data:", round(roc_auc_score(y_test, y_pred_prob),2))

draw_roc(y_test, y_pred_prob)

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [4,8,10],
    'min_samples_leaf': range(100, 400, 200),
    'min_samples_split': range(200, 500, 200),
    'n_estimators': [100,200, 300], 
    'max_features': [5, 10]
}

# Create a based model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, scoring='roc_auc', cv = 3, n_jobs = -1,verbose = 1)


In [ ]:
# fit model
grid_search.fit(X_train, y_train)

In [ ]:
# print best hyperparameters
print("Best AUC-ROC Score on train data: ", grid_search.best_score_)
print("Best hyperparameters: ", grid_search.best_params_)

In [ ]:
rf_tuned=RandomForestClassifier(max_depth= 10, max_features= 10, min_samples_leaf= 100, 
                           min_samples_split= 200, n_estimators= 200)
rf_tuned

In [ ]:
rf_tuned.fit(X_train, y_train)

# predict churn on test data
y_pred = rf_tuned.predict(X_test)

# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)

#get scores
getScores(confusion)

# check area under curve
y_pred_prob = rf_tuned.predict_proba(X_test)[:, 1]
print("AUC-Roc Score on test data:", round(roc_auc_score(y_test, y_pred_prob),2))

draw_roc(y_test, y_pred_prob)

In [ ]:
print(classification_report(y_test,y_pred))

### Model4 - XGBoost

Let's finally try XGBoost. The hyperparameters are the same, some important ones being ```subsample```, ```learning_rate```, ```max_depth``` etc.


In [ ]:
# fit model on training data with default hyperparameters
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
# make predictions for test data
# use predict_proba since we need probabilities to compute auc
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred=model.predict(X_test)

In [ ]:
# evaluate predictions
# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)

#get scores
getScores(confusion)

# check area under curve
print("AUC-Roc Score on test data:", round(roc_auc_score(y_test, y_pred_prob),2))

draw_roc(y_test, y_pred_prob)

__Hyperparameter tuning__

In [ ]:
# specify range of hyperparameters
param_grid = {'learning_rate': [0.2, 0.6], 
             'subsample': [0.3, 0.6, 0.9]}          


# specify model
xgb_model = XGBClassifier(max_depth=2, n_estimators=200)

# set up GridSearchCV()
model_cv = GridSearchCV(estimator = xgb_model, 
                        param_grid = param_grid, 
                        scoring= 'roc_auc', 
                        cv = 3, 
                        verbose = 1,
                        return_train_score=True)      



In [ ]:
# fit the model
model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head()

In [ ]:
# convert parameters to int for plotting on x-axis
cv_results['param_learning_rate'] = cv_results['param_learning_rate'].astype('float')
# cv_results['param_max_depth'] = cv_results['param_max_depth'].astype('float')
cv_results.head()

In [ ]:
# # plotting
plt.figure(figsize=(16,6))

param_grid = {'learning_rate': [0.2, 0.6], 
             'subsample': [0.3, 0.6, 0.9]} 


for n, subsample in enumerate(param_grid['subsample']):
    

    # subplot 1/n
    plt.subplot(1,len(param_grid['subsample']), n+1)
    df = cv_results[cv_results['param_subsample']==subsample]

    plt.plot(df["param_learning_rate"], df["mean_test_score"])
    plt.plot(df["param_learning_rate"], df["mean_train_score"])
    plt.xlabel('learning_rate')
    plt.ylabel('AUC')
    plt.title("subsample={0}".format(subsample))
    plt.ylim([0.60, 1])
    plt.legend(['test score', 'train score'], loc='upper left')
    plt.xscale('log')

The results show that a subsample size of 0.6 and learning_rate of about 0.2 seems optimal. 
Also, XGBoost has resulted in the highest ROC AUC obtained (across various hyperparameters). 


Let's build a final model with the chosen hyperparameters.

In [ ]:
# chosen hyperparameters
# 'objective':'binary:logistic' outputs probability rather than label, which we need for auc
params = {'learning_rate': 0.2,
          'max_depth': 2, 
          'n_estimators':200,
          'subsample':0.9,
         'objective':'binary:logistic'}

# fit model on training data
model = XGBClassifier(params = params)
model.fit(X_train, y_train)

In [ ]:
# predict
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]
#The first column in y_pred is the P(0), i.e. P(not fraud), and the second column is P(1/fraud).

In [ ]:
# evaluate predictions
# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)

#get scores
getScores(confusion)

# check area under curve
print("AUC-Roc Score on test data:", round(roc_auc_score(y_test, y_pred_prob),2))

draw_roc(y_test, y_pred_prob)

In [ ]:
# feature importance
importance = dict(zip(high_value_cust.columns, model.feature_importances_))
importance